Item 29 Avoid More Than Two Control Subexpressions in Comprehensions

Things to Remember
- Assignment expressions make it possible for comprehensions and generator expressions to reuse the value from one condition elsewhere in the same comprehension, which can improve readability and performance.
- Although it's possible to use an assignment expression outside of a comprehension or generator expression's condition, you should avoid doing so.   

In [ ]:
# - you need to verify that a request is sufficiently in stock 
#   and above minimum threshold for shipping (in batches of 8) 
stock = {
    'nails': 125,
    'screws': 35,
    'wingnuts': 8,
    'washers': 24
}

order = ['screws', 'wingnuts', 'clips']

def get_batches(count, size):
    return count // size

result = {}
for name in order:
    count = stock.get(name, 0)
    batches = get_batches(count, 8)
    if batches:
        result[name]= batches
print(result)


In [ ]:
# using dictionary comprehension
found = {name: get_batches(stock.get(name, 0), 8)
         for name in order
         if get_batches(stock.get(name, 0), 8)}
print(found)

what are the problems with the above approach?
- get_batches(stock.get(name, 0), 8) expression is repeated in two places
- this adds visual noise and hurts readability
- increase the likelihood of introducing a bug if the expression in two places aren't kept in sync 

In [ ]:
# solution 
# - using walrus operator, :=, to form an assignment expression
#   as part of the comprehension 

found = {name: batches for name in order
         if (batches := get_batches(stock.get(name, 0), 8))}
print(found)

In [ ]:
# - you can define an assignment expression 
#   in the value expression for a comprehension
# - however, you might get an exception at runtime
#   because of the order in which comprehensions are
#   evaluated

result = {name: (tenth := count // 10) # value expression
          for name, count in stock.items() if tenth > 0} # error  

In [ ]:
# solution - move the assignment expression into the condition
result = {name: tenth for name, count in stock.items()
          if (tenth := count // 10) > 0} # move the assignment expression here
print(result)

loop variable leakage
- happens when using the walrus operator in the value part of the comprehension that does not have a condition
- the loop variable will leak into the containing scope
- this is similar to the loop variable leakage in a normal for loop
- leaking loop variables is bad so use assignment expressions only in the condition part of a comprehension

In [ ]:
half = [(last := count // 2) for count in stock.values()] # using := and there is no condition
print(f'Last time of {half} is {last}') # loop variable leakage

In [ ]:
for count in stock.values():
    pass
print(f'Last time of {list(stock.values())} is {count}') # loop variable leakage

In [ ]:
# leakage won't happen for the loop variables from comprehensions
half = [count2 // 2 for count in stock.values()]
print(half)
print(count2) # error

In [ ]:
# use assignment expressions with generator 
found = ((name, batches) for name in order
         if(batches := get_batches(stock.get(name, 0), 8)))
print(next(found))
print(next(found))